# Simulation

We add balls, always to the same location. Set gravity and physics, and add obstacles (the "segments" which is both the bar and the wall on the sides).

It can still happen that a beam moves after it is flipped (it collides with wall, and is reflected), but ideal this happenning is very rare. This issue could be further refined by changing the shape of the bars.

In [ ]:
import sys, random
import pygame
import pymunk
import pymunk.pygame_util
import pygame as pg
import numpy as np

def add_ball(space):
    """Add a ball to the given space at a random position"""
    mass = 6
    radius = 5
    inertia = pymunk.moment_for_circle(mass, 0, radius, (0,0))
    body = pymunk.Body(mass, inertia)
    x = 300
    body.position = x, 50
    shape = pymunk.Circle(body, radius, (0,0))
    shape.friction = 0
    shape.color = pg.color.THECOLORS['green']

    space.add(body, shape)
    return shape


def add_segment(orientation,row,column,space):
    """Adds the flippable beams.It is an arrow like shape, so the top is heavier, and the arrow should stuck
    to the wall, which is also generated here. A PinJoint is set so the bar can rotate around."""
    rotation_center_body = pymunk.Body(body_type = pymunk.Body.STATIC)
    rotation_center_body.position = (0,0)
    body = pymunk.Body()
    body.position = (0,0)
    
    xc=300+26*(column-5)
    y=100+75*row
    if orientation=='LEFT':
        y1=y-15
        y2=y+15
        l1 = pymunk.Poly(body, [(xc-25,y-25),
                                (xc-20,y-25),
                                (xc+25,y+25),
                                (xc-25,y-20)])

    else:
        y1=y+15
        y2=y-15
        l1 = pymunk.Poly(body, [(xc+25,y-25),
                                (xc+20,y-25),
                                (xc-25,y+25),
                                (xc+25,y-20)])
    
    l1.friction = 20000000
    l1.mass = 5
    l1.elasticity=0.0
    l1.momentum=0.
    l1.color = pg.color.THECOLORS['red']

    rotation_center_joint = pymunk.PinJoint(body, rotation_center_body, (xc,y), (xc,y))
    
    side1 = pymunk.Segment(space.static_body, (xc-25.05,y+35), (xc-25.05,y-35), 1)
    side1.color = pg.color.THECOLORS['lightblue']
    side2 = pymunk.Segment(space.static_body, (xc+25.05,y+35), (xc+25.05,y-35), 1)
    side2.color = pg.color.THECOLORS['lightblue']
    side1.friction=2000000
    side2.friction=2000000
    side1.elasticity=0.0
    side1.mass = 1000000
    side2.mass = 1000000
    space.add(side1)
    space.add(side2)
    
    space.add(l1, body, rotation_center_joint)
    return l1

def add_scoring(space):
    """method to add the scoring board. Only walls."""
    for x in [175,200,225,250,275,300,325,350,375,400,425]:
        side=pymunk.Segment(space.static_body, (x,500), (x,450), 1)
        side.color = pg.color.THECOLORS['lightblue']
        space.add(side)
        
from PIL import Image

def main():
    pygame.init()
    screen = pygame.display.set_mode((600, 500))
    pygame.display.set_caption("Memico")
    clock = pygame.time.Clock()

    space = pymunk.Space()
    space.gravity = (0.0, 300.0)
    space.damping=1.0

    add_scoring(space)
    #The commented lines below create a random initial state.
    #startsegment=add_segment(np.random.choice(['LEFT','RIGHT']),0,5,space)
    #add_segment(np.random.choice(['LEFT','RIGHT']),1,4,space)
    #add_segment(np.random.choice(['LEFT','RIGHT']),1,6,space)

    #add_segment(np.random.choice(['LEFT','RIGHT']),2,3,space)
    #add_segment(np.random.choice(['LEFT','RIGHT']),2,5,space)
    #add_segment(np.random.choice(['LEFT','RIGHT']),2,7,space)

    #add_segment(np.random.choice(['LEFT','RIGHT']),3,2,space)
    #add_segment(np.random.choice(['LEFT','RIGHT']),3,4,space)
    #add_segment(np.random.choice(['LEFT','RIGHT']),3,6,space)
    #add_segment(np.random.choice(['LEFT','RIGHT']),3,8,space)

    #add_segment(np.random.choice(['LEFT','RIGHT']),4,1,space)
    #add_segment(np.random.choice(['LEFT','RIGHT']),4,3,space)
    #add_segment(np.random.choice(['LEFT','RIGHT']),4,5,space)
    #add_segment(np.random.choice(['LEFT','RIGHT']),4,7,space)
    #add_segment(np.random.choice(['LEFT','RIGHT']),4,9,space)
    
    #The code below creates an initial state with all bars looking left.
    startsegment=add_segment('LEFT',0,5,space)
    add_segment('LEFT',1,4,space)
    add_segment('LEFT',1,6,space)

    add_segment('LEFT',2,3,space)
    add_segment('LEFT',2,5,space)
    add_segment('LEFT',2,7,space)

    add_segment('LEFT',3,2,space)
    add_segment('LEFT',3,4,space)
    add_segment('LEFT',3,6,space)
    add_segment('LEFT',3,8,space)

    add_segment('LEFT',4,1,space)
    add_segment('LEFT',4,3,space)
    add_segment('LEFT',4,5,space)
    add_segment('LEFT',4,7,space)
    add_segment('LEFT',4,9,space)


    images=[]
    for _ in range(10):
        balls = []
        draw_options = pymunk.pygame_util.DrawOptions(screen)

        ticks_to_next_ball = 50
        flag=True
        while flag:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    sys.exit(0)
                elif event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
                    sys.exit(0)

            ticks_to_next_ball -= 1
            if ticks_to_next_ball <= 0:
                ticks_to_next_ball = 250
                ball_shape = add_ball(space)
                balls.append(ball_shape)

            screen.fill((255,255,255))

            balls_to_remove = []
            for ball in balls:
                if ball.body.position.y > 550:
                    balls_to_remove.append(ball)
                    flag=False

            for ball in balls_to_remove:
                space.remove(ball, ball.body)
                balls.remove(ball)

            space.debug_draw(draw_options)
            strFormat = 'RGBA'
            #uncomment this if images to be saved
            raw_str = pygame.image.tostring(screen, strFormat, False)
            image = Image.frombytes(strFormat, (screen.get_size()), raw_str)
            images.append(image)
            space.step(1/50.0)

            pygame.display.flip()
            clock.tick(50)
    flag=False
    #uncomment this if images to be saved to gif
    images[0].save('memicolowNew.gif',
                                save_all=True, append_images=images[1:],
                                optimize=True, duration=len(images)*(1/50.0)/2, loop=0)


main()